In [338]:
!pip install pypdfium2
import pypdfium2 as pyp
import os
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


[nltk_data] Downloading package punkt to /Users/saurabh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saurabh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [339]:
data_dir = 'Praj Industries'

In [340]:
# find all the pdf files in the current directory
pdf_files = [f for f in os.listdir(data_dir) if f.endswith('.pdf')]
# print(len(pdf_files))
# empty df
df = pd.DataFrame(columns=['file', 'text'])
title = []
texts = []
for pdf_file in pdf_files:
    # create a Pdf object
    pdf = pyp.PdfDocument(os.path.join(data_dir, pdf_file))
    pages = len(pdf)
    text = []
    title.append(pdf_file)
    for i in range(pages):
        raw = pdf[i].get_textpage().get_text_range()
        words = word_tokenize(raw)
        words = [word.lower() for word in words]
        text.extend(words)
    print(text)
    texts.append(text)
df['file'] = title
df['text'] = texts

['praj', 'industries', 'limited', ',', '‘', 'praj', 'tower', '’', ',', '274', '&', '275/2', ',', 'bhumkar', 'chowk', ',', 'hinjewadi', 'road', ',', 'hinjewadi', ',', 'pune', '411057', '.', 'india', 'cin', ':', 'l27101pn1985plc038031', 'ph', '.', ':', '+91-20-22941000/71802000', 'fax', ':', '+91-20-22941116', 'info', '@', 'praj.net', 'www.praj.net', 'date', ':', '7th', 'november', ',', '2023', 'ref', '.', ':', 'pil/dvn/l121/2023-24', 'security', 'code', 'no', '.', ':', '522205', 'company', 'code-', 'prajind', 'bse', 'limited', 'phiroze', 'jeejeebhoy', 'towers', ',', '25th', 'floor', ',', 'dalal', 'street', ',', 'mumbai', '-', '400', '001', 'fax', ':', '022-', '22723121/3719/2037/2039/2041/2061', 'national', 'stock', 'exchange', 'of', 'india', 'ltd.', 'exchange', 'plaza', ',', '5th', 'floor', ',', 'plot', 'no', '.', 'c/1', ',', 'g', 'block', ',', 'bandra-kurla', 'complex', ',', 'bandra', '(', 'east', ')', ',', 'mumbai', '-', '400', '051', 'fax', ':', '022', '–', '66418124/25/26', ',', '2

In [341]:
# remove stop words and punctuation
stop_words = set(stopwords.words('english'))
# adding some extra stop words manually, based on the data
stop_words.add('praj')
stop_words.add('industries')
stop_words.add('limited')
df['text'] = df['text'].apply(lambda x: [word for word in x if word.lower() not in stop_words and (word.isalpha() or '-' in word)])

In [342]:
# create inverted index for df
inverted_index = {}
for i, row in df.iterrows():
    for word in row['text']:
        if word not in inverted_index:
            inverted_index[word] = set()
        inverted_index[word].add(i)
print(len(inverted_index))

4728


In [343]:
# add biword index to inverted index, take only top 100 biwords
biwords = {}
for i, row in df.iterrows():
    for j in range(len(row['text'])-1):
        biword = row['text'][j] + ' ' + row['text'][j+1]
        if biword not in biwords:
            biwords[biword] = set()
        biwords[biword].add(i)
biwords = sorted(biwords.items(), key=lambda x: len(x[1]), reverse=True)[:100]
for biword, docs in biwords:
    if biword not in inverted_index:
        inverted_index[biword] = set()
    inverted_index[biword].update(docs)

In [344]:
len(inverted_index)

4828

In [358]:
# implement a simple boolean retrieval model with OR and AND operations between words
# remove and and or from stop words if they are present
if 'and' in stop_words:
    stop_words.remove('and')

if 'or' in stop_words:
    stop_words.remove('or')

def get_query(word,inverted_index):
    unigram = word_tokenize(word)
    bigram = [unigram[i] + ' ' + unigram[i+1] for i in range(len(unigram)-1)]
    if len(bigram):
        unigram = []
    result = set()
    for word in unigram + bigram:
        if word in inverted_index:
            result = result.union(inverted_index[word])
    return result
            
    
def search(query,inverted_index,df):
    query = query.lower()
    query = [word for word in query.split() if word not in stop_words and len(word)>=2]
    # first break the query into words, use AND and OR to seperate the words
    words = []
    tmp = ''
    for word in query:
        if word in ['and', 'or']:
            words.append(tmp)
            words.append(word)
            tmp = ''
        else:
            tmp += word + ' '
    tmp = tmp[:-1]
    words.append(tmp)
    
    result_set = set()
    for i,word in enumerate(words):
        if word != 'or' and word != 'and':
            result = get_query(word,inverted_index)
            if i == 0:
                result_set = result
            else:
                if words[i-1] == 'or':
                    result_set = result_set.union(result)
                else:
                    result_set = result_set.intersection(result)
    
    return df.iloc[list(result_set)]['file'].sort_values()

In [369]:
query = 'order book and bio energy and sugar'
result = search(query,inverted_index,df)
print(len(result))
result

8


9     Praj Industries  investor presentation August ...
7     Praj Industries  investor presentation August ...
8     Praj Industries  investor presentation Februar...
6     Praj Industries  investor presentation June 20...
4     Praj Industries  investor presentation March 2...
5     Praj Industries  investor presentation january...
10         Praj Industries Transcript February 2024.pdf
0          Praj Industries Transcript November 2023.pdf
Name: file, dtype: object